In [142]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import unidecode
import re

In [143]:
chotot_df = pd.read_csv("chotot.csv")

from copy import deepcopy
chotot_df.hang = chotot_df.hang.replace(["Vinfast","Reult"], ["VinFast","Renault"])
all_hang = deepcopy(chotot_df.hang.unique())
print(all_hang)
all_hang = np.append(all_hang,['Thaco','McLaren','Cuu Long','Dongfeng','Dongben','Lamborghini','Vinaxuki','TMT',"Hoa Mai","Fuso","Proton","Chien Thang","Do Thanh","Giai Phong","Hino","Howo","JAC","Jinbei","JRD","Kenbo","Samco","Teraco","Komatsu","Veam","Thanh Cong","Truong Giang"])

all_dong_xe = chotot_df.dong_xe.unique()
# Giá
chotot_gia = []
import math
for value in chotot_df.gia:
    chotot_gia.append(int(value.replace(".","").replace("đ","0")))
chotot_df['gia'] = chotot_gia
chotot_df[chotot_df['gia'] == 0]

# Kiểu dáng
chotot_df.kieu_dang = chotot_df.kieu_dang.replace(["Van","Minivan (MPV)","Coupe (2 cửa)","Pick-up (bán tải)"],["Van/Minivan", "Van/Minivan","Coupe","Bán tải / Pickup"])

chotot_df.info()
chotot_df.head()

['Toyota' 'Kia' 'Ford' 'Chevrolet' 'Audi' 'Hyundai' 'Mitsubishi' 'VinFast'
 'Honda' 'Lexus' 'Mazda' 'Mekong' 'Mercedes Benz' 'Suzuki' 'Daewoo'
 'Isuzu' 'Nissan' 'Changan' 'Peugeot' nan 'BMW' 'Jaguar' 'Renault' 'MG'
 'Daihatsu' 'LandRover' 'Hãng khác' 'Porsche' 'Subaru' 'Volkswagen'
 'Baic' 'Jeep' 'Mini' 'Zotye' 'Asia' 'Acura' 'Bentley' 'BYD' 'Scion'
 'Ssangyong' 'Chery' 'Luxgen' 'Haima' 'Fiat' 'Cadillac' 'Volvo' 'Proton'
 'Ferrari' 'Citroen' 'Samsung' 'Lincoln' 'Lifan' 'Geely' 'SYM' 'Maserati'
 'Rolls Royce' 'UAZ' 'Infiniti' 'Aston Martin' 'Gaz' 'Lada' 'Dodge'
 'Alfa Romeo' 'Man' 'Buick' 'GMC' 'Tesla' 'Maybach' 'Chrysler' 'Smart'
 'Opel']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30673 entries, 0 to 30672
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   dong_xe      30472 non-null  object
 1   gia          30673 non-null  int64 
 2   hang         30473 non-null  object
 3   hop_so       30465 non-null  obje

,dong_xe,gia,hang,hop_so,kieu_dang,km_da_di,nam_sx,nhien_lieu,so_cho_ngoi,tinh_trang,xuat_xu
0,Corona,74000000,Toyota,Số sàn,Sedan,289569,1993,Xăng,5,Đã sử dụng,Nhật Bản
1,Sorento,1230000000,Kia,Tự động,SUV / Cross over,3100,2021,Dầu,7,Đã sử dụng,Việt Nam
2,Cerato,513000000,Kia,Tự động,Sedan,39000,2018,Xăng,5,Đã sử dụng,NaN
3,Explorer,1786000000,Ford,Tự động,SUV / Cross over,300000,2018,Xăng,8,Đã sử dụng,Mỹ
4,Spark,130000000,Chevrolet,Tự động,Hatchback,15088,2008,Xăng,5,Đã sử dụng,NaN


In [144]:
bonbanh_df = pd.read_csv("bonbanh.csv")
bonbanh_df = bonbanh_df.drop(columns=['kieu_dang'])
bonbanh_df = bonbanh_df.rename(columns={"gia" : "hon_hop"})
bonbanh_df = bonbanh_df.rename(columns={"dong_xe" : "kieu_dang"})
bonbanh_df.head()

,kieu_dang,hon_hop,hop_so,km_da_di,mau_xe,nhien_lieu,so_cho_ngoi,so_cua,tinh_trang,xuat_xu
0,Sedan,Xe\tMercedes Benz S class\tS450L Luxury\t2019\...,Số tự động,"9,000 Km",Đen,Xăng\t3.0 L,5 chỗ,4 cửa,Xe đã dùng,Lắp ráp trong nước
1,Sedan,Xe\tMercedes Benz C class\tC200\t2017\t- 1 Tỷ ...,Số tự động,"20,000 Km",Đỏ,Xăng\t2.0 L,5 chỗ,4 cửa,Xe đã dùng,Lắp ráp trong nước
2,Crossover,Xe\tHyundai Kona\t2.0 AT\t2021\t- 579 Triệu,Số tự động,0 Km,Đỏ,Xăng\t2.0 L,5 chỗ,5 cửa,Xe mới,Lắp ráp trong nước
3,Crossover,Xe\tHyundai Tucson\t2.0 AT Đặc biệt\t2021\t- 8...,Số tự động,0 Km,Trắng,Xăng\t2.0 L,5 chỗ,5 cửa,Xe mới,Lắp ráp trong nước
4,Sedan,Xe\tPorsche Panamera\t3.0 V6\t2017\t- 5 Tỷ 750...,Số tự động,"32,000 Km",Đen,Xăng\t3.0 L,4 chỗ,5 cửa,Xe đã dùng,Nhập khẩu


Xóa các bản ghi có chứa trường có giá trị rỗng

In [145]:
chotot_df = chotot_df.dropna()
bonbanh_df = bonbanh_df.dropna()
chotot_df.head()

,dong_xe,gia,hang,hop_so,kieu_dang,km_da_di,nam_sx,nhien_lieu,so_cho_ngoi,tinh_trang,xuat_xu
0,Corona,74000000,Toyota,Số sàn,Sedan,289569,1993,Xăng,5,Đã sử dụng,Nhật Bản
1,Sorento,1230000000,Kia,Tự động,SUV / Cross over,3100,2021,Dầu,7,Đã sử dụng,Việt Nam
3,Explorer,1786000000,Ford,Tự động,SUV / Cross over,300000,2018,Xăng,8,Đã sử dụng,Mỹ
6,Sedona,990000000,Kia,Tự động,Van/Minivan,40000,2019,Dầu,7,Đã sử dụng,Việt Nam
8,Triton,320000,Mitsubishi,Tự động,Bán tải / Pickup,123,2010,Dầu,5,Đã sử dụng,Thái Lan


Xóa dấu, xóa dấu cách, đưa về chữ in thường

In [146]:
def remove_accent(text):
    return unidecode.unidecode(text)
def preprocess_remove_unicode(ten):
    ten = remove_accent(ten)
    ten="".join(ten.split())
    ten = ten.lower()
    return ten

Xử lý xuất xứ

In [147]:
list_xuat_xu = chotot_df['xuat_xu'].tolist()
def process_origin(i):
    i=preprocess_remove_unicode(i)
    if "viet" in i:
        return "Lắp ráp trong nước"
    else: 
        return "Nhập khẩu"
for i,value in enumerate(list_xuat_xu):
    list_xuat_xu[i] = process_origin(value)


In [148]:
chotot_df.drop(columns='xuat_xu')
chotot_df['xuat_xu'] = list_xuat_xu

In [149]:
chotot_df.head()

,dong_xe,gia,hang,hop_so,kieu_dang,km_da_di,nam_sx,nhien_lieu,so_cho_ngoi,tinh_trang,xuat_xu
0,Corona,74000000,Toyota,Số sàn,Sedan,289569,1993,Xăng,5,Đã sử dụng,Nhập khẩu
1,Sorento,1230000000,Kia,Tự động,SUV / Cross over,3100,2021,Dầu,7,Đã sử dụng,Lắp ráp trong nước
3,Explorer,1786000000,Ford,Tự động,SUV / Cross over,300000,2018,Xăng,8,Đã sử dụng,Nhập khẩu
6,Sedona,990000000,Kia,Tự động,Van/Minivan,40000,2019,Dầu,7,Đã sử dụng,Lắp ráp trong nước
8,Triton,320000,Mitsubishi,Tự động,Bán tải / Pickup,123,2010,Dầu,5,Đã sử dụng,Nhập khẩu


In [150]:
columns = ['dong_xe','gia','hang','hop_so','kieu_dang','km_da_di','nam_sx','nhien_lieu','so_cho_ngoi','tinh_trang','xuat_xu']


Xử lý chính tả, gộp các hãng bonbanh

In [151]:
def process_branch_bonbanh(i):
    i = preprocess_remove_unicode(i)
    if i == 'pronto':
        return "Proton"
    elif "chienthang" in i:
        return "Chien Thang"
    elif "cuul" in i:
        return "Cuu Long"
    elif "dothanh" in i:
        return "Do Thanh"
    elif "giaiph" in i:
        return "Giai Phong"
    elif "hino" in i:
        return "Hino"
    elif "howo" in i:
        return "Howo"
    elif "jac" in i:
        return "JAC"
    elif "jinbei" in i:
        return "Jinbei"
    elif "jrd" in i:
        return "JRD"
    elif "kenbo" in i:
        return "Kenbo"
    elif "samco" in i:
        return "Samco"
    elif "tera" in i:
        return "Teraco"
    elif "koma" in i:
        return "Komatsu"
    elif "kuma" in i:
        return "Komatsu"
    elif "thanhcong" in i:
        return "Thanh Cong"
    elif "truonggiang" in i:
        return "Truong Giang"
    elif "veam" in i:
        return "Veam"
    # elif "reult" in i:
    #     return "Renault"
    else: 
        return "Hãng khác"

In [152]:
# tách hon_hop thành giá, năm sản xuất, hãng
x = set()
gia = []
nam_sx = []
hang = []
dong_xe = []

for idx, value in enumerate(bonbanh_df['hon_hop']):
    nam, gia_tien = value.split("\t")[-2:]
    nam_sx.append(nam)

    gia_tien = gia_tien.split()[1:]
    gia_tri = []
    for idx, gt in enumerate(gia_tien):
        if gt == "Triệu":
            gia_tri.append(int(gia_tien[idx-1])*1e6)
        if gt == "Tỷ":
            gia_tri.append(int(gia_tien[idx-1])*1e9)
    gia_tri = int(sum(gia_tri))
    gia.append(gia_tri)

    hang_xe = value.split("\t")[1:-2][0].split()
    # for i in hang_xe:
    #     i = process_branch_bonbanh(i)
    bonbanh_hangxe = ""
    flag = False
    for i in hang_xe:
        bonbanh_hangxe += i
        if bonbanh_hangxe in all_hang:
            #print(bonbanh_hangxe)
            hang.append(bonbanh_hangxe)
            flag = True
            break
        else:
            bonbanh_hangxe += " "
    if flag == False:
        hang.append(bonbanh_hangxe)
    
    dong = hang_xe[len(bonbanh_hangxe.split()):]
    dong_xe_tmp = ""
    if len(dong) > 0:
        for i in dong:
            dong_xe_tmp += i
            if dong_xe_tmp in all_dong_xe:
                #print(bonbanh_hangxe)
                dong_xe.append(dong_xe_tmp)
                dong_xe_tmp = ""
                break
            else:
                bonbanh_hangxe += " "
        if len(dong_xe_tmp) > 1:
            dong_xe.append(dong_xe_tmp)
    else:
        dong_xe.append(float('NaN'))
for i,value in enumerate(hang):
    if value not in all_hang:
        hang[i] = process_branch_bonbanh(value)

# nhiên liệu
nhien_lieu = []
for idx, value in enumerate(bonbanh_df['nhien_lieu']):
    nhien_lieu.append(value.split()[0])

# so_cho_ngoi
so_cho_ngoi = []
for idx, value in enumerate(bonbanh_df['so_cho_ngoi']):
    so_cho_ngoi.append(value.split()[0])

# tình trang
bonbanh_df["tinh_trang"] = bonbanh_df['tinh_trang'].replace(['Xe đã dùng','Xe mới'],['Đã sử dụng',"Mới"])

# km đã đi
km_da_di = []
for idx, value in enumerate(bonbanh_df['km_da_di']):
    km_da_di.append(int(value.split()[0].replace(",","")))
    
bonbanh_df.hop_so = bonbanh_df["hop_so"].replace(["Số tự động"], "Tự động")
bonbanh_df["km_da_di"] = km_da_di
bonbanh_df["so_cho_ngoi"] = so_cho_ngoi
bonbanh_df["nhien_lieu"] = nhien_lieu
bonbanh_df["nam_sx"] = nam_sx
bonbanh_df["gia"] = gia
bonbanh_df["dong_xe"] = dong_xe
bonbanh_df = bonbanh_df.drop(columns=['hon_hop','mau_xe',"so_cua"])
bonbanh_df["hang"] = hang
bonbanh_df.kieu_dang = bonbanh_df.kieu_dang.replace(["Crossover","SUV"],["SUV / Cross over", "SUV / Cross over"])
bonbanh_df


,kieu_dang,hop_so,km_da_di,nhien_lieu,so_cho_ngoi,tinh_trang,xuat_xu,nam_sx,gia,dong_xe,hang
0,Sedan,Tự động,9000,Xăng,5,Đã sử dụng,Lắp ráp trong nước,2019,4480000000,Sclass,Mercedes Benz
1,Sedan,Tự động,20000,Xăng,5,Đã sử dụng,Lắp ráp trong nước,2017,1155000000,Cclass,Mercedes Benz
2,SUV / Cross over,Tự động,0,Xăng,5,Mới,Lắp ráp trong nước,2021,579000000,Kona,Hyundai
3,SUV / Cross over,Tự động,0,Xăng,5,Mới,Lắp ráp trong nước,2021,825000000,Tucson,Hyundai
4,Sedan,Tự động,32000,Xăng,4,Đã sử dụng,Nhập khẩu,2017,5750000000,Panamera,Porsche
...,...,...,...,...,...,...,...,...,...,...,...
35969,SUV / Cross over,Tự động,0,Xăng,5,Đã sử dụng,Lắp ráp trong nước,2020,870000000,CX5,Mazda
35970,Sedan,Số tay,0,Xăng,5,Đã sử dụng,Lắp ráp trong nước,2001,185000000,Camry,Toyota
35971,Sedan,Tự động,8000,Xăng,5,Đã sử dụng,Lắp ráp trong nước,2020,529000000,Accent,Hyundai
35972,SUV / Cross over,Tự động,120000,Xăng,5,Đã sử dụng,Nhập khẩu,2005,495000000,RX,Lexus


In [153]:
bonbanh_df.hang.unique()

array(['Mercedes Benz', 'Hyundai', 'Porsche', 'Toyota', 'BMW',
       'LandRover', 'Honda', 'Ford', 'Mazda', 'Thaco', 'Mitsubishi',
       'Lexus', 'Kia', 'Audi', 'MG', 'Peugeot', 'Jaguar', 'Suzuki',
       'VinFast', 'Chevrolet', 'Rolls Royce', 'Daewoo', 'Nissan', 'Volvo',
       'Cadillac', 'Mini', 'Subaru', 'Bentley', 'Ferrari', 'Isuzu',
       'Hãng khác', 'Lincoln', 'Volkswagen', 'McLaren', 'Alfa Romeo',
       'Do Thanh', 'Daihatsu', 'Hoa Mai', 'Jeep', 'Vinaxuki', 'Fiat',
       'Kenbo', 'Cuu Long', 'Dongfeng', 'Dongben', 'Renault',
       'Chien Thang', 'Samco', 'Luxgen', 'Ssangyong', 'TMT', 'Giai Phong',
       'Truong Giang', 'Veam', 'SYM', 'UAZ', 'Chrysler', 'Maserati',
       'Lamborghini', 'Infiniti', 'Lada', 'Chery', 'Zotye', 'Jinbei',
       'Fuso', 'Hino', 'Baic', 'Howo', 'Haima', 'JAC', 'Asia', 'Smart',
       'Teraco', 'Lifan', 'Changan', 'Samsung', 'Komatsu', 'Acura',
       'Opel', 'Mekong', 'Proton', 'Dodge', 'JRD', 'Thanh Cong', 'Gaz',
       'GMC', 'Scion'], dtype

Gộp 2 dữ liệu 

In [154]:
frames = [bonbanh_df,chotot_df]
data = pd.concat(frames)
data = data.reset_index()
data = data.drop('index', axis=1)

In [155]:
index1 = data[ data['hop_so'] == '-' ].index
data.drop(index1 , inplace=True)

In [156]:
index2 = data[ data['nhien_lieu'] == '-' ].index
data.drop(index2 , inplace=True)

In [157]:
data['km_da_di'] = data['km_da_di'].replace(['70 000 - 74 999'],'70000')

In [158]:
data.dong_xe = data.dong_xe.replace(["khác"], "Dòng khác")

In [159]:
list_dong_xe = []
list_dong_xe1 = data.dong_xe.tolist()
# print(list_dong_xe)
for i in list_dong_xe1:
    i = str(i)
    if i == "Dòng khác":
        list_dong_xe.append(i)
    # print(type(i))
    else:
        i = "".join(i.split())
        if "Class" in i:
            i = i.replace("Class","class")
            i = "".join(i.split())
        list_dong_xe.append(i)

        

In [160]:
for i in range(len(list_dong_xe)):
    if len(list_dong_xe[i]) < 4:
        list_dong_xe[i] = list_dong_xe[i].upper()
    if "TẤN" in list_dong_xe[i]:
        list_dong_xe[i] = list_dong_xe[i].replace("TẤN","tấn")
    elif "Tấn" in list_dong_xe[i]:
        list_dong_xe[i] = list_dong_xe[i].replace("Tấn","tấn")
    elif "tấn" in list_dong_xe[i]:
        pass
    elif list_dong_xe[i] == "Dòng khác":
        pass
    else:
        list_dong_xe[i] = re.sub(r"[^a-zA-Z0-9]","",list_dong_xe[i])
# list1 = set()
# for i in list_dong_xe:
#     i = preprocess_remove_unicode(i)
#     list1.add(i)
# print(len(list1))
# print(list1)
# print(list_dong_xe)

In [161]:
data.drop(columns='dong_xe')
data['dong_xe'] = list_dong_xe

In [162]:
data

,kieu_dang,hop_so,km_da_di,nhien_lieu,so_cho_ngoi,tinh_trang,xuat_xu,nam_sx,gia,dong_xe,hang
0,Sedan,Tự động,9000,Xăng,5,Đã sử dụng,Lắp ráp trong nước,2019,4480000000,Sclass,Mercedes Benz
1,Sedan,Tự động,20000,Xăng,5,Đã sử dụng,Lắp ráp trong nước,2017,1155000000,Cclass,Mercedes Benz
2,SUV / Cross over,Tự động,0,Xăng,5,Mới,Lắp ráp trong nước,2021,579000000,Kona,Hyundai
3,SUV / Cross over,Tự động,0,Xăng,5,Mới,Lắp ráp trong nước,2021,825000000,Tucson,Hyundai
4,Sedan,Tự động,32000,Xăng,4,Đã sử dụng,Nhập khẩu,2017,5750000000,Panamera,Porsche
...,...,...,...,...,...,...,...,...,...,...,...
54991,Van/Minivan,Số sàn,80000,Xăng,8,Đã sử dụng,Lắp ráp trong nước,2019,565000000,Innova,Toyota
54992,Sedan,Tự động,38000,Xăng,5,Đã sử dụng,Lắp ráp trong nước,2018,563000000,3,Mazda
54993,Hatchback,Tự động,70000,Xăng,5,Đã sử dụng,Lắp ráp trong nước,2009,185000000,Morning,Kia
54994,Sedan,Số sàn,2555,Xăng,5,Đã sử dụng,Nhập khẩu,1999,42500000,Matiz,Daewoo


In [163]:
data.dong_xe.unique().tolist()
# print(list1)
# print(len(list1))

['Sclass',
 'Cclass',
 'Kona',
 'Tucson',
 'Panamera',
 'Rush',
 '5',
 '3',
 'SantaFe',
 'GLS',
 'I10',
 'RangeRover',
 'CRV',
 'Ranger',
 'CX5',
 'Corolla',
 'Ollin',
 'Xpander',
 '6',
 'RX',
 'Innova',
 'Sprinter',
 'Morning',
 'Vios',
 'City',
 'A4',
 'Fortuner',
 'Carnival',
 'EcoSport',
 'Camry',
 'HS',
 'LandCruiser',
 '5008',
 'Cerato',
 'Wigo',
 '911',
 'Eclass',
 'Zace',
 'GLC',
 'CX8',
 'FPace',
 'Ertiga',
 'Attrage',
 'K3',
 'ZS',
 'SuperCarryVan',
 'A6',
 'LS',
 'LUX',
 'Everest',
 'Spark',
 'Elantra',
 'Cayenne',
 'Cullinan',
 'Ghost',
 'Accent',
 'Pajero',
 'Getz',
 'Swift',
 'Transit',
 'Trailblazer',
 'I20',
 'F150',
 '3008',
 'Zinger',
 'Outlander',
 'GLAclass',
 'Matiz',
 'Grandlivina',
 'Hilux',
 'Brio',
 'Soluto',
 'Prado',
 'Jazz',
 'Civic',
 'Tourneo',
 'Yaris',
 'XC90',
 'Tiida',
 'LX',
 'Escalade',
 'Cooper',
 'SuperCarryTruck',
 '2',
 'Vitara',
 'Q5',
 'X5',
 'GLKclass',
 'Sorento',
 'Optima',
 'Fadil',
 'Forester',
 'Navara',
 'Terra',
 'GrandStarex',
 'Triton

In [164]:
data.to_csv('mix_data.csv', sep=',', encoding='utf-8')